In [51]:
# !pip install --user xmltodict

In [1]:
import random
import pytz
import shapely
from shapely.geometry import Point
import elasticsearch
from datetime import datetime
import time


In [2]:
# define mapping
mapping = {
        "mappings": {
            #"mydocs": {
                "properties": {
                    "Date": {
                     "type": "date"
                    },
                    "Destination": {
                     "type": "geo_point"
                    },
                    "Origin": {
                      "type": "geo_point"
                    },
                    "Title": {
                      "type": "keyword"
                    },
                }
            #}
       }
}

In [6]:
# load Elasticsearch
try:
    es = elasticsearch.Elasticsearch(["kooplex-fiek.elte.hu"],port=9100)
except:
    print( "unable to es")

In [7]:
# create index mass_traffic
try:
    response = es.indices.create(index="random_point2",body=mapping, ignore=400)
except Exception as e:
    print(e)

In [8]:
response

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'random_point2'}

In [6]:
now = pytz.utc.localize(datetime.utcnow())
id=1
latr1, latr2, lonr1, lonr2 = [random.uniform(-10,10) for i in range(4)]
while True:
    try:
        new_body = {}
        new_body["Date"] =  datetime.strftime(now,'%Y-%m-%dT%H:%M:%S.%fZ')
        p = Point((random.uniform(0.2+latr1,23+latr1), random.uniform(45.5+lonr1,49+lonr1))) 
        new_body["Destination"] = { "lat": p.x, 'lon': p.y}                
        p = Point((random.uniform(-43+latr2,-10+latr2), random.uniform(25.5+lonr2,39+lonr2))) 
        new_body["Origin"] = {"lat": p.x, 'lon': p.y}              

        new_body["Title"] = random.choice(['flight1, flight2, flight3'])
        id+=1


        print(new_body)
        res = es.index(index="random_point", doc_type="_doc", id=id, body=new_body)
        print( "------------------------------------------")
        print(res)
    except Exception as e:
        print( "------------------------------------------")
        print("unable to index in elasticsearch")
        print( "------------------------------------------")
        error = "{0}: {1}".format(id,e)
        print( error)
        print( "------------------------------------------")
    print( "all done, starting again soon")
    time.sleep(5)
    if id==30:
        id=1
        latr1, latr2, lonr1, lonr2 = random(4)
        latr1, latr2, lonr1, lonr2 = [random.uniform(-10,10) for i in range(4)]


{'Date': '2020-11-18T22:22:46.992122Z', 'Destination': {'lat': 2.446666040740263, 'lon': 39.691025283524425}, 'Origin': {'lat': -40.997995434117186, 'lon': 33.4908122609994}, 'Title': 'flight1, flight2, flight3'}
------------------------------------------
{'_index': 'random_point', '_type': '_doc', '_id': '2', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 245, '_primary_term': 1}
all done, starting again soon
{'Date': '2020-11-18T22:22:46.992122Z', 'Destination': {'lat': 15.635578954524437, 'lon': 40.177133412237296}, 'Origin': {'lat': -15.880408809584328, 'lon': 26.700175268293563}, 'Title': 'flight1, flight2, flight3'}
------------------------------------------
{'_index': 'random_point', '_type': '_doc', '_id': '3', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 246, '_primary_term': 1}
all done, starting again soon


KeyboardInterrupt: 

## Queries DSL

GET random_point/_search
{
   "query": {
    "bool": {
      "must": {
        "match_all": {}
      },
      "filter": {
        "geo_bounding_box": {
          "Origin": {
            "top_left": {
              "lat": -28.6603547,
              "lon": 0.2268691
            },
            "bottom_right": {
              "lat": -48.3658642,
              "lon": -26.9556643
            }
          }
        }
      }
    }
  }
}



In [15]:
# Search in random_point index
res = es.search(index="random_point")
res

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 30, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'random_point',
    '_type': '_doc',
    '_id': '2',
    '_score': 1.0,
    '_source': {'Date': '2020-11-19T09:42:49.600762Z',
     'Destination': {'lat': 14.128594028987003, 'lon': 44.258336663067276},
     'Origin': {'lat': -18.99753070156272, 'lon': 18.649682090504445},
     'Title': 'flight1, flight2, flight3'}},
   {'_index': 'random_point',
    '_type': '_doc',
    '_id': '4',
    '_score': 1.0,
    '_source': {'Date': '2020-11-19T09:42:49.600762Z',
     'Destination': {'lat': 29.771240013385356, 'lon': 45.80298806015347},
     'Origin': {'lat': -25.890167699195864, 'lon': 25.649076820110572},
     'Title': 'flight1, flight2, flight3'}},
   {'_index': 'random_point',
    '_type': '_doc',
    '_id': '3',
    '_score': 1.0,
    '_source': {'Date': '2020-11-19T09:42:49.600762Z',
     'Dest

In [17]:
res=es.get(index='random_point',doc_type='_doc', id=2)
res

{'_index': 'random_point',
 '_type': '_doc',
 '_id': '2',
 '_version': 6,
 '_seq_no': 276,
 '_primary_term': 1,
 'found': True,
 '_source': {'Date': '2020-11-19T09:42:49.600762Z',
  'Destination': {'lat': 14.128594028987003, 'lon': 44.258336663067276},
  'Origin': {'lat': -18.99753070156272, 'lon': 18.649682090504445},
  'Title': 'flight1, flight2, flight3'}}